In [ ]:
from classiq import (
    Analyzer,
    GeneratedCircuit,
    Model,
    RegisterUserInput,
    execute,
    show,
    synthesize,
)
from classiq.builtin_functions import Adder, StatePreparation
from classiq.execution import (
    AzureBackendPreferences,
    ClassiqBackendPreferences,
    ExecutionDetails,
    ExecutionPreferences,
    set_quantum_program_execution_preferences,
)
from classiq.model import Constraints, Preferences
from classiq.synthesis import (
    set_constraints,
    set_execution_preferences,
    set_preferences,
)

In [ ]:
# defining probabilities
prob_a = [0.5, 0, 0, 0.5]
prob_b = [0, 0.5, 0.5, 0]

# defining state preparation
sp_a = StatePreparation(
    probabilities=prob_a, error_metric={"KL": {"upper_bound": 0.01}}
)
sp_b = StatePreparation(
    probabilities=prob_b, error_metric={"KL": {"upper_bound": 0.01}}
)

# defining the adder
adder = Adder(
    left_arg=RegisterUserInput(size=2),
    right_arg=RegisterUserInput(size=2),
)

# initiating a model
model = Model()

# wiring state preparations
a = model.StatePreparation(params=sp_a)
b = model.StatePreparation(params=sp_b)

# wiring the adder
adder_out = model.Adder(
    params=adder, in_wires={"left_arg": a["OUT"], "right_arg": b["OUT"]}
)
print(adder_out)

# setting the outputs
model.set_outputs(
    {"a": adder_out["left_arg"], "b": adder_out["right_arg"], "sum": adder_out["sum"]}
)
model.sample()
backend_preferences = ClassiqBackendPreferences(
    backend_name="aer_simulator_statevector"
)
serialized_model = model.get_model()
serialized_model = set_execution_preferences(
    serialized_model,
    execution_preferences=ExecutionPreferences(backend_preferences=backend_preferences),
)

In [ ]:
# fixing the seed
seed = 206755496
preferences = Preferences(random_seed=seed)
serialized_model = set_preferences(serialized_model, preferences=preferences)


# synthesizing the first circuit
constraints = Constraints(optimization_parameter="depth")
serialized_model = set_constraints(serialized_model, constraints=constraints)
qprog = synthesize(serialized_model)


# printing the output mapping
circuit = GeneratedCircuit.from_qprog(qprog)
print(circuit.data.qubit_mapping.logical_outputs)

# vizualizing the circuit
show(qprog)

# save results
circuit.save_results("my_arithmetic_circuit.json")

# save qasm file
# my_file = open("my_arithmetic_circuit.qasm", "w").write(circuit.qasm)


# executing the first circuit with a simulator
from classiq import execute

results_raw = execute(qprog).result()

results = results_raw[0].value
print(results.counts)


# output results
output_results = results.parsed_counts
print(output_results)

# loading results from IDE executor
# my_downloaded_results = ExecutionDetails.parse_file(
#     "my_ionq_simulator_arithmetic_circuit_execusion_results.json"
# )
# print(my_downloaded_results.counts)


# motivation for shalower circuit
print(
    "circuit width: ",
    circuit.data.width,
    " circuit depth: ",
    circuit.transpiled_circuit.depth,
)

# synthesizing the second circuit
constraints = Constraints(optimization_parameter="width")
serialized_model_optimized_for_width = set_constraints(
    serialized_model, constraints=constraints
)
qprog_optimized_for_width = synthesize(serialized_model_optimized_for_width)
circuit_optimized_for_width = GeneratedCircuit.from_qprog(qprog_optimized_for_width)
print(
    "circuit width: ",
    circuit_optimized_for_width.data.width,
    " circuit depth: ",
    circuit_optimized_for_width.transpiled_circuit.depth,
)

# analyzing the 2nd circuit
analyzer = Analyzer(circuit=circuit_optimized_for_width)
analyzer.get_hardware_comparison_table(["Azure Quantum"])
analyzer.plot_hardware_comparison_table()

# viewing the 2nd and 3rd circuits
show(qprog_optimized_for_width)
show(qprog)